# 3x3 Rubiks' Cube Solver

Imports:

In [437]:
from rubiks_cube import rubiks_cube as rc
import numpy as np
from copy import deepcopy as copy
from hashlib import sha1

We first make a solved cube:

In [438]:
cube = rc.Cube()
cube.print()

rubiks_cube: 

      b b b
      b b b
      b b b
      -----
o o o|w w w|r r r|y y y
o o o|w w w|r r r|y y y
o o o|w w w|r r r|y y y
      -----
      g g g
      g g g
      g g g



In [439]:
cube2 = copy(cube)

In [440]:
cube2.turn_face('R')

In [441]:
cube2.print()
cube.print()

rubiks_cube: 

      b b w
      b b w
      b b w
      -----
o o o|w w g|r r r|b y y
o o o|w w g|r r r|b y y
o o o|w w g|r r r|b y y
      -----
      g g y
      g g y
      g g y

rubiks_cube: 

      b b b
      b b b
      b b b
      -----
o o o|w w w|r r r|y y y
o o o|w w w|r r r|y y y
o o o|w w w|r r r|y y y
      -----
      g g g
      g g g
      g g g



We can get a one-hot representation of the cube like this:

In [442]:
cube2.turn_face('R')
cube2.get_binary_array(one_hot=True).reshape(6, 8, 6)

array([[[1, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0],
        [0, 0, 0, 1, 0, 0],
        [1, 0, 0, 0, 0, 0]],

       [[0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 1, 0, 0, 0, 0]],

       [[0, 0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0, 0]],

       [[0, 0, 0, 1, 0, 0],
        [0, 0, 0, 1, 0, 0],
        [0, 0, 0, 1, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0],
        [0, 0, 0, 1, 0, 0],
        [1, 0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 1, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 

## Detecting a G1 state

In [443]:
def top_bottom_CO_Edge(bin_arr, relative_to=0):
    '''Whether the cube has all corners oriented and top/bottom edges oriented, relative to the top and bottom faces indexed as relative_to and relative_to + 3'''
    i = relative_to
    for face in (bin_arr[i*8:i*8+8], bin_arr[(i+3)*8:(i+3)*8+8]):
        for piece in face:
            if not piece[i] and not piece[i+3]:
                return False

    return True

In [444]:
def mid_EO(bin_arr, relative_to=0):
    '''Whether the M slice edges are oriented, relative to the top and bottom faces indexed as relative_to and relative_to + 3'''
    if relative_to == 0:
        ind_to_check = ((1, 4), (1, 2), (4, 2), (4, 4))
        valid_faces = (1, 4)
    elif relative_to == 1:
        ind_to_check = ((0, 4), (0, 2), (3, 4), (3, 2))
        valid_faces = (0, 3)
    else:
        ind_to_check = ((4, 3), (4, 1), (1, 3), (1, 1))
        valid_faces = (1, 4)
    
    for (i, j) in ind_to_check:
        piece = bin_arr[i*8+j]
        if not piece[valid_faces[0]] and not piece[valid_faces[1]]:
            return False
        
    return True

In [445]:
def is_in_G1(cube, relative_to=0):
    bin_arr = cube.get_binary_array(one_hot=True).reshape(-1, 6)
    return top_bottom_CO_Edge(bin_arr, relative_to=relative_to) and mid_EO(bin_arr, relative_to=relative_to)

In [446]:
def is_solved(cube):
    return cube.is_solved()

## Brute force a first stage

We start with an arbitrary scramble:

In [447]:
scramble = copy(cube)
scramble.scramble("R F L2")
# scramble.scramble("D B U' R' U R2 L' D F U2 F D2 R2 F2 R2 F2 B' D2 L2 U2 L'")
scramble.print()

rubiks_cube: 

      g b w
      g b w
      y b w
      -----
y y b|r w g|r r r|b y o
o o o|y w g|r r r|b y w
o o o|y o o|w w g|r r w
      -----
      b g g
      b g g
      b y y



In [448]:
is_in_G1(scramble)

False

In [449]:

face_move_map = {'U': 0, 'F': 1, 'R': 2, 'D': 3, 'B': 4, 'L': 5}

valid_moves = [('U', 0), ('U', 1), ('U', 2), 
    ('F', 0), ('F', 1), ('F', 2), 
    ('R', 0), ('R', 1), ('R', 2), 
    ('D', 0), ('D', 1), ('D', 2), 
    ('B', 0), ('B', 1), ('B', 2),
    ('L', 0), ('L', 1), ('L', 2)]

def next_valid_moves(prev_moves):
    if len(prev_moves) == 0:
        return valid_moves

    (last_face, _) = prev_moves[-1]
    last_ind = face_move_map[last_face]

    if len(prev_moves) > 1:
        (last2_face, _) = prev_moves[-2]
        last2_ind = face_move_map[last2_face]

        # WLOG, if you spin R then L, you don't want to do either again
        if abs(last2_ind - last_ind) == 3:
            ind = last_ind % 3
            return valid_moves[0:ind*3] + valid_moves[ind*3+3:(ind+3)*3] + valid_moves[(ind+3)*3+3:]

    return valid_moves[0:last_ind*3] + valid_moves[last_ind*3+3:]

all_180_moves = [('U', 2), 
    ('F', 2),
    ('R', 2), 
    ('D', 2), 
    ('B', 2), 
    ('L', 2)]

other_ways = [('U', 0), ('U', 1), 
    ('F', 0), ('F', 1), 
    ('R', 0), ('R', 1), 
    ('D', 0), ('D', 1), 
    ('B', 0), ('B', 1),
    ('L', 0), ('L', 1)]

def valid_g1_moves(relative_to=0):
    valid_g1s = all_180_moves + other_ways[relative_to*2:relative_to*2+2] + other_ways[(relative_to+3)*2:(relative_to+3)*2+2]
    return valid_g1s

In [450]:
def turn_face(cube, move):
    next_cube = copy(cube)
    (face, way) = move
    next_cube.turn_face(face, way)
    return next_cube

In [451]:
def solve_to_G1(scr_cube):
    cubes_queue = [(scr_cube, [])]

    num_moves = 0
    i = 0

    while(True):
        (cube, prev_moves) = cubes_queue.pop(0)

        for rel in (0, 1, 2):
            if is_in_G1(cube, rel):
                print("Num moves: " + str(len(prev_moves)) + "\t Iter: " + str(i))
                return prev_moves
        
        if num_moves < len(prev_moves):
            num_moves = len(prev_moves)
            print("Now doing " + str(num_moves) + " moves, " + str(i) + " iters")
        
        for move in next_valid_moves(prev_moves):
            next_cube = turn_face(cube, move)
            cubes_queue.append((next_cube, prev_moves + [move]))

        i = i+1

In [452]:
g1_sol = solve_to_G1(scramble)
print(g1_sol)

Now doing 1 moves, 1 iters
Now doing 2 moves, 19 iters
Now doing 3 moves, 289 iters
Num moves: 3	 Iter: 4024
[('L', 2), ('F', 1), ('R', 0)]


This is _super_ slow :/

## Pruning tables: pt 2

We now explore solving a cube in a G1 state, using a pruning table that stores how close we are to a solution.

In [453]:
def hash_cube(cube):
    return hash(str(cube.get_binary_array()))

In [464]:
def calc_g1_movecount():
    G1_moves = valid_g1_moves(0)
    g1_sols = dict()
    q = [(copy(rc.Cube()), [])]
    max_moves = 5

    while(len(q) > 0):
        (cube, prev_moves) = q.pop(0)

        cube_hash = hash_cube(cube)

        if cube_hash not in g1_sols:
            g1_sols[cube_hash] = len(prev_moves)
            
            for move in G1_moves:
                next_cube = turn_face(cube, move)
                q.append((next_cube, prev_moves + [move]))
    
        if len(prev_moves) > max_moves:
            return g1_sols
    return g1_sols

g1_sols = calc_g1_movecount()

In [465]:
len(g1_sols)

3614

In [456]:
def recover_sol(cube, g1_sols):
    G1_moves = valid_g1_moves(0)
    best_candidate = []
    shortest_len = 9001

    if cube.is_solved():
        return []
    
    cube.print()

    for move in G1_moves:
        cube_to_try = turn_face(cube, move)
        hash_val = hash_cube(cube_to_try)
        if hash_val in g1_sols:
            print(move)
            print(g1_sols[hash_val])
            # candidate = recover_sol(cube_to_try, g1_sols)
            # if(len(candidate) < shortest_len):
            #     best_candidate = [move] + candidate

    return best_candidate

In [459]:
cube5 = copy(cube)
cube5.scramble("F2")
print(hash_cube(cube5) in g1_sols)
print(g1_sols[hash_cube(cube5)])
recover_sol(cube5, g1_sols)

True
1
rubiks_cube: 

      b b b
      b b b
      b b b
      -----
o o o|w w w|r r r|y y y
o o o|w w w|r r r|y y y
r r r|y y y|o o o|w w w
      -----
      g g g
      g g g
      g g g

('F', 2)
0


[]